# Tarea 2

In [ ]:
import cv2  
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

In [ ]:
gris = cv2.cvtColor(cv2.imread('mandril.jpg'), cv2.COLOR_BGR2GRAY)
ggris = cv2.GaussianBlur(gris, (3, 3), 0)

sobelx = cv2.Sobel(ggris, cv2.CV_64F, 1, 0)
sobely = cv2.Sobel(ggris, cv2.CV_64F, 0, 1)
sobel = cv2.add(sobelx, sobely)

# 2. Convertir a 8 bits
sobel8 = cv2.convertScaleAbs(sobel)

# 3. Umbralizado sobre Sobel
valorUmbral = 100  # puedes ajustar
_, sobelUmbral = cv2.threshold(sobel8, valorUmbral, 255, cv2.THRESH_BINARY)

# 4. Conteo por filas y columnas
conteoFilas = np.count_nonzero(sobelUmbral, axis=1)
conteoCols = np.count_nonzero(sobelUmbral, axis=0)

# 5. Calcular máximos y umbrales
maxFila = np.max(conteoFilas)
maxCol = np.max(conteoCols)

umbralFila = 0.9 * maxFila
umbralCol = 0.9 * maxCol

# 6. Detectar filas y columnas destacadas
filasDestacadas = np.where(conteoFilas >= umbralFila)[0]
colsDestacadas = np.where(conteoCols >= umbralCol)[0]

print(f"Máximo filas = {maxFila}, Máximo columnas = {maxCol}")
print(f"Filas destacadas: {filasDestacadas}")
print(f"Columnas destacadas: {colsDestacadas}")

# 7. Dibujar sobre la imagen original
imgMarcada = gris.copy()
imgMarcada = cv2.cvtColor(imgMarcada, cv2.COLOR_GRAY2BGR)  # pasar a color para pintar

for f in filasDestacadas:
    cv2.line(imgMarcada, (0, f), (imgMarcada.shape[1], f), (0,0,255), 1)  # rojo

for c in colsDestacadas:
    cv2.line(imgMarcada, (c, 0), (c, imgMarcada.shape[0]), (0,255,0), 1)  # verde

# Mostrar resultados
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.title("Sobel umbralizado")
plt.imshow(sobelUmbral, cmap='gray')
plt.axis("off")

plt.subplot(1,2,2)
plt.title("Filas y columnas destacadas (Sobel)")
plt.imshow(imgMarcada)
plt.axis("off")
plt.show()

Tarea 3

In [ ]:
cap = cv2.VideoCapture(0)

modo = 0   # 0: normal, 1: inverso, 2: reducción bits
bits = 8   # profundidad inicial (máx 8 bits por canal)

print("Controles:")
print(" - Tecla 'm' para cambiar de modo")
print(" - Tecla 'a' para reducir bits (solo en modo 2)")
print(" - Tecla 'd' para aumentar bits (solo en modo 2)")
print(" - Tecla 'q' para salir")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    if modo == 0:
        # Modo normal
        salida = frame.copy()

    elif modo == 1:
        # Modo inverso
        salida = cv2.bitwise_not(frame)

    elif modo == 2:
        # Modo reducción de bits
        factor = 2 ** bits
        salida = np.floor(frame / (256/factor)) * (256/factor)
        salida = salida.astype(np.uint8)
        cv2.putText(salida, f"Bits: {bits}", (10,30), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0,255,0), 2, cv2.LINE_AA)

    # Mostrar resultado
    cv2.imshow("Demostrador", salida)

    k = cv2.waitKey(1) & 0xFF
    if k == ord('q'):
        break
    elif k == ord('m'):
        modo = (modo + 1) % 3  # cambiar modo
    elif k == ord('a'):  # disminuir bits
        if modo == 2 and bits > 1:
            bits -= 1
    elif k == ord('d'):  # aumentar bits
        if modo == 2 and bits < 8:
            bits += 1

cap.release()
cv2.destroyAllWindows()